In [23]:
# Import required libraries
import requests
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import base64
import time
import re
from bs4 import BeautifulSoup
from datetime import datetime
import urllib.parse

In [24]:
# Connection credentials
REED_API_KEY = "ff3b8ce7-3225-4336-911f-3b1ecad843ec"

# Azure PostgreSQL
DB_HOST = "dvdrental.postgres.database.azure.com"
DB_NAME = "postgres"
DB_USER = "gbadmin"
DB_PASSWORD = "Catherin3!"
DB_PORT = 5432

# Reed API authentication
auth_string = f"{REED_API_KEY}:"
encoded_auth = base64.b64encode(auth_string.encode()).decode()
reed_headers = {'Authorization': f'Basic {encoded_auth}'}

print("✅ Configuration loaded")

✅ Configuration loaded


# Daily Job Update - Reed API to Azure PostgreSQL

This notebook fetches 100 new Data Analyst jobs from Reed.co.uk API daily and adds them to the Azure PostgreSQL database, avoiding duplicates.

**How it works:**
1. Connect to Azure PostgreSQL and get existing job IDs
2. Fetch 100+ jobs from Reed API
3. Filter out jobs already in the database
4. Extract skills from new jobs
5. Insert only new jobs (up to 100) into the database

In [25]:
# Step 1: Connect to Azure PostgreSQL and get existing job IDs
print("🔄 Connecting to Azure PostgreSQL...")

try:
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT,
        sslmode='require',
        connect_timeout=10
    )
    cursor = conn.cursor()
    print("✅ Connected to Azure PostgreSQL")
    
    # Get all existing job IDs to avoid duplicates
    cursor.execute("SELECT jobid FROM jobs;")
    existing_job_ids = set(row[0] for row in cursor.fetchall())
    print(f"📊 Found {len(existing_job_ids)} existing jobs in database")
    
except Exception as e:
    print(f"❌ Connection error: {e}")
    existing_job_ids = set()
    conn = None

🔄 Connecting to Azure PostgreSQL...
✅ Connected to Azure PostgreSQL
📊 Found 99 existing jobs in database
✅ Connected to Azure PostgreSQL
📊 Found 99 existing jobs in database


In [26]:
# Step 2: Search for recent Data Analyst jobs from Reed API
print("🔄 Fetching jobs from Reed API...")

def search_reed_jobs(keywords='data analyst', results_to_take=100, skip=0):
    """Search Reed.co.uk for jobs"""
    search_url = 'https://www.reed.co.uk/api/1.0/search'
    params = {
        'keywords': keywords,
        'resultsToTake': results_to_take,
        'resultsToSkip': skip
    }
    
    response = requests.get(search_url, headers=reed_headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data['results'], data['totalResults']
    else:
        print(f"Error: {response.status_code}")
        return [], 0

# Fetch jobs - get more than 100 to ensure we have enough new ones
jobs_batch_1, total = search_reed_jobs('data analyst', results_to_take=100, skip=0)
jobs_batch_2, _ = search_reed_jobs('data analyst', results_to_take=100, skip=100)

all_searched_jobs = jobs_batch_1 + jobs_batch_2
print(f"📥 Fetched {len(all_searched_jobs)} jobs from API")
print(f"   (Total available: {total})")

# Filter out jobs already in database
new_jobs = [job for job in all_searched_jobs if job['jobId'] not in existing_job_ids]
print(f"✨ Found {len(new_jobs)} NEW jobs (not in database)")

# Take only the first 100 new jobs
new_jobs_to_add = new_jobs[:100]
print(f"📌 Will add {len(new_jobs_to_add)} new jobs to database")

🔄 Fetching jobs from Reed API...
📥 Fetched 200 jobs from API
   (Total available: 2558)
✨ Found 194 NEW jobs (not in database)
📌 Will add 100 new jobs to database
📥 Fetched 200 jobs from API
   (Total available: 2558)
✨ Found 194 NEW jobs (not in database)
📌 Will add 100 new jobs to database


In [27]:
# Step 3: Fetch FULL details for new jobs
print("🔄 Fetching full job details...")

def get_job_details(job_id):
    """Get full details for a specific job"""
    url = f'https://www.reed.co.uk/api/1.0/jobs/{job_id}'
    response = requests.get(url, headers=reed_headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching job {job_id}: {response.status_code}")
        return None

new_jobs_full = []
for i, job in enumerate(new_jobs_to_add):
    if i % 10 == 0:
        print(f"  Progress: {i}/{len(new_jobs_to_add)}...")
    
    details = get_job_details(job['jobId'])
    if details:
        new_jobs_full.append(details)
    
    time.sleep(0.1)  # Be nice to the API

print(f"✅ Fetched full details for {len(new_jobs_full)} jobs")

🔄 Fetching full job details...
  Progress: 0/100...
  Progress: 10/100...
  Progress: 10/100...
  Progress: 20/100...
  Progress: 20/100...
  Progress: 30/100...
  Progress: 30/100...
  Progress: 40/100...
  Progress: 40/100...
  Progress: 50/100...
  Progress: 50/100...
  Progress: 60/100...
  Progress: 60/100...
  Progress: 70/100...
  Progress: 70/100...
  Progress: 80/100...
  Progress: 80/100...
  Progress: 90/100...
  Progress: 90/100...
✅ Fetched full details for 100 jobs
✅ Fetched full details for 100 jobs


In [28]:
# Step 4: Extract skills from job descriptions
print("🔄 Extracting skills from job descriptions...")

# Skills mapping (same as in scraping.ipynb)
skills_mapping = {
    'Python': ['python', 'py'],
    'R': [r'\br\b', r'\br programming\b'],
    'SQL': ['sql', 'mysql', 'postgresql', 'postgres', 't-sql', 'pl/sql', 'mssql'],
    'SAS': ['sas'],
    'VBA': ['vba', 'visual basic'],
    'Tableau': ['tableau'],
    'Power BI': ['power bi', 'powerbi', 'power-bi'],
    'Looker': ['looker'],
    'Qlik': ['qlik', 'qliksense', 'qlikview'],
    'Excel': ['excel', 'advanced excel', 'ms excel', 'spreadsheet'],
    'AWS': ['aws', 'amazon web services'],
    'Azure': ['azure', 'microsoft azure'],
    'GCP': ['gcp', 'google cloud'],
    'MongoDB': ['mongodb', 'mongo'],
    'Oracle': ['oracle database', 'oracle db'],
    'SQL Server': ['sql server', 'mssql', 'microsoft sql'],
    'Snowflake': ['snowflake'],
    'Pandas': ['pandas'],
    'NumPy': ['numpy'],
    'Spark': ['spark', 'pyspark', 'apache spark'],
    'ETL': ['etl'],
    'Airflow': ['airflow', 'apache airflow'],
    'Statistics': ['statistics', 'statistical analysis', 'statistical modeling'],
    'Machine Learning': ['machine learning', 'ml'],
    'A/B Testing': ['a/b test', 'ab test'],
    'Git': ['git', 'github', 'gitlab'],
    'Jira': ['jira'],
    'API': ['api', 'rest api', 'restful'],
}

def extract_skills_improved(text):
    """Extract skills from job description"""
    if pd.isna(text):
        return []
    
    # Clean HTML tags
    if '<' in text and '>' in text:
        soup = BeautifulSoup(text, 'html.parser')
        text = soup.get_text()
    
    found_skills = set()
    text_lower = text.lower()
    
    for skill_name, patterns in skills_mapping.items():
        for pattern in patterns:
            regex = r'\b' + pattern.replace(' ', r'\s+') + r'\b'
            if re.search(regex, text_lower, re.IGNORECASE):
                found_skills.add(skill_name)
                break
    
    return list(found_skills)

# Create DataFrame and extract skills
df_new_jobs = pd.DataFrame(new_jobs_full)
df_new_jobs['skills'] = df_new_jobs['jobDescription'].apply(extract_skills_improved)
df_new_jobs['skills_list'] = df_new_jobs['skills'].apply(lambda x: ', '.join(x) if len(x) > 0 else '')

print(f"✅ Extracted skills from {len(df_new_jobs)} jobs")
print(f"\nSample of new jobs:")
print(df_new_jobs[['jobId', 'jobTitle', 'employerName', 'locationName']].head())

🔄 Extracting skills from job descriptions...
✅ Extracted skills from 100 jobs

Sample of new jobs:
      jobId                 jobTitle                     employerName  \
0  56107225             Data Analyst                        NG Bailey   
1  56107954             Data Analyst                      esure Group   
2  55988131             Data Analyst                  Smart4Chemicals   
3  56094030             Data Analyst  Marley Risk Consultants Limited   
4  56038552  Data Analyst Apprentice                               QA   

     locationName  
0           Leeds  
1         Reigate  
2           Wigan  
3  Bishops Cleeve  
4       Edgbaston  
✅ Extracted skills from 100 jobs

Sample of new jobs:
      jobId                 jobTitle                     employerName  \
0  56107225             Data Analyst                        NG Bailey   
1  56107954             Data Analyst                      esure Group   
2  55988131             Data Analyst                  Smart4Chemicals

In [29]:
# Step 5: Clean and transform data (match existing table structure)
print("🔄 Preparing data for database insertion...")

def clean_new_jobs(df):
    """Apply same transformations as in column_changes_to_jobs.ipynb"""
    df_clean = df.copy()
    
    # Lowercase all column names FIRST
    df_clean.columns = df_clean.columns.str.lower()
    
    # Drop columns not in the database
    columns_to_remove = [
        "minimumsalary", "maximumsalary", "currency", "salarytype", 
        "salary", "externalurl", "parttime", "skills"
    ]
    existing_cols_to_remove = [col for col in columns_to_remove if col in df_clean.columns]
    if existing_cols_to_remove:
        df_clean = df_clean.drop(columns=existing_cols_to_remove)
        print(f"   Dropped columns: {existing_cols_to_remove}")
    
    # Convert expirationdate and add daysremaining
    if 'expirationdate' in df_clean.columns:
        df_clean['expirationdate'] = pd.to_datetime(df_clean['expirationdate'], format='%d/%m/%Y', errors='coerce')
        df_clean['daysremaining'] = (df_clean['expirationdate'] - datetime.now()).dt.days
        df_clean['expirationdate'] = df_clean['expirationdate'].dt.strftime('%Y-%m-%d')
    
    # Convert fulltime to text
    if 'fulltime' in df_clean.columns:
        df_clean['fulltime'] = df_clean['fulltime'].replace({True: 'Full time', False: 'Part time'})
    
    # Clean HTML from job descriptions and remove invalid Unicode
    def clean_text(text):
        if pd.notna(text):
            # Remove HTML tags
            soup = BeautifulSoup(text, 'html.parser')
            clean = soup.get_text()
            # Remove surrogate pairs and other problematic Unicode characters
            clean = clean.encode('utf-8', errors='ignore').decode('utf-8')
            return clean
        return text
    
    df_clean['jobdescription'] = df_clean['jobdescription'].apply(clean_text)
    
    # Also clean other text fields that might have Unicode issues
    text_columns = ['jobtitle', 'employername', 'locationname', 'contracttype']
    for col in text_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].apply(
                lambda x: x.encode('utf-8', errors='ignore').decode('utf-8') if pd.notna(x) else x
            )
    
    return df_clean

df_new_jobs_clean = clean_new_jobs(df_new_jobs)
print(f"✅ Data cleaned and transformed")
print(f"   Rows: {len(df_new_jobs_clean)}")
print(f"   Columns: {list(df_new_jobs_clean.columns)}")


🔄 Preparing data for database insertion...
   Dropped columns: ['minimumsalary', 'maximumsalary', 'currency', 'salarytype', 'salary', 'externalurl', 'parttime', 'skills']
✅ Data cleaned and transformed
   Rows: 100
   Columns: ['employerid', 'employername', 'jobid', 'jobtitle', 'locationname', 'yearlyminimumsalary', 'yearlymaximumsalary', 'dateposted', 'expirationdate', 'joburl', 'fulltime', 'contracttype', 'jobdescription', 'applicationcount', 'skills_list', 'daysremaining']


In [30]:
# Step 6: Insert new jobs into Azure PostgreSQL database
if conn and not df_new_jobs_clean.empty:
    try:
        print("🔄 Inserting new jobs into database...")
        
        # Use SQLAlchemy for easier insertion
        password_encoded = urllib.parse.quote_plus(DB_PASSWORD)
        connection_string = f"postgresql://{DB_USER}:{password_encoded}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=require"
        engine = create_engine(connection_string, connect_args={'connect_timeout': 30})
        
        # Insert new jobs (append to existing table)
        df_new_jobs_clean.to_sql('jobs', engine, if_exists='append', index=False, method='multi', chunksize=100)
        
        print(f"✅ Successfully inserted {len(df_new_jobs_clean)} new jobs into database!")
        
        # Verify total count
        cursor.execute("SELECT COUNT(*) FROM jobs;")
        total_jobs = cursor.fetchone()[0]
        print(f"📊 Total jobs in database: {total_jobs}")
        
        engine.dispose()
        
    except Exception as e:
        print(f"❌ Error inserting into database: {e}")
        import traceback
        traceback.print_exc()
else:
    if not conn:
        print("⚠️  No database connection")
    if df_new_jobs_clean.empty:
        print("⚠️  No new jobs to insert")

🔄 Inserting new jobs into database...
✅ Successfully inserted 100 new jobs into database!
📊 Total jobs in database: 200
✅ Successfully inserted 100 new jobs into database!
📊 Total jobs in database: 200


In [31]:
# Step 7: Save backup to CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f"new_jobs_backup_{timestamp}.csv"
df_new_jobs_clean.to_csv(csv_filename, index=False)
print(f"💾 Backup saved to {csv_filename}")

💾 Backup saved to new_jobs_backup_20251127_172642.csv


In [32]:
# Close database connection
if conn:
    cursor.close()
    conn.close()
    print("✅ Database connection closed")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"✅ Found {len(new_jobs)} new jobs from Reed API")
print(f"✅ Inserted {len(df_new_jobs_clean)} jobs into database")
print(f"✅ Backup saved to {csv_filename}")
print("="*60)

✅ Database connection closed

SUMMARY
✅ Found 194 new jobs from Reed API
✅ Inserted 100 jobs into database
✅ Backup saved to new_jobs_backup_20251127_172642.csv
